## Backpropagación



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class NeuralNetwork(nn.Module):
    def __init__(self, num_inputs, num_hidden, num_outputs):
        super(NeuralNetwork, self).__init__()
        self.hidden_layer = nn.Linear(num_inputs, num_hidden)
        self.output_layer = nn.Linear(num_hidden, num_outputs)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        hidden = self.sigmoid(self.hidden_layer(x))
        output = self.sigmoid(self.output_layer(hidden))
        return output

# Cambiando el nombre de la instancia para evitar confusión
model = NeuralNetwork(num_inputs=2, num_hidden=2, num_outputs=2)

# Optimizador y función de pérdida
optimizer = optim.SGD(model.parameters(), lr=0.5)
criterion = nn.MSELoss()  # Instancia correctamente MSELoss

# Datos de entrenamiento
inputs = torch.tensor([[0.05, 0.1]], dtype=torch.float32)
targets = torch.tensor([[0.01, 0.99]], dtype=torch.float32)

# Bucle de entrenamiento
for i in range(10000):
    model.zero_grad()           # Resetea los gradientes
    outputs = model(inputs)     # Pase hacia adelante
    loss = criterion(outputs, targets)  # Calcula la pérdida
    loss.backward()             # Pase hacia atrás
    optimizer.step()            # Actualiza los parámetros

    if i % 1000 == 0:
        print(f'Epoch {i}, Loss: {loss.item()}')

# Código opcionalmente añadido para validación/pruebas
# Supongamos que tenemos datos de validación:
validation_inputs = torch.tensor([[0.1, 0.2]], dtype=torch.float32)
validation_targets = torch.tensor([[0.05, 0.95]], dtype=torch.float32)

# Evaluar el modelo en modo de evaluación
model.eval()
with torch.no_grad():  # Desactiva el cálculo de gradientes
    validation_outputs = model(validation_inputs)
    validation_loss = criterion(validation_outputs, validation_targets)
    print(f'Pérdida de validación: {validation_loss.item()}')


### Ejercicios

**Ejercicio 1:** Implementación manual del algoritmo  de retropropagación

Crea una red neuronal simple (por ejemplo, una red con una capa oculta) desde cero en Python. Implementa manualmente el algoritmo de retropropagación para actualizar los pesos de la red.  Utiliza esta red para entrenar un modelo que pueda clasificar puntos en dos clases basadas en su posición en un plano 2D. Genera datos sintéticos que sean linealmente separables.

**Ejercicio 2:**  Comparación de funciones de activación

Modifica la red del ejercicio 1 para probar diferentes funciones de activación (ReLU, sigmoidal, tanh) en la capa oculta.  Compara el rendimiento del entrenamiento con cada función de activación. Observa cómo cambia la rapidez de convergencia y si alguna configuración es particularmente propensa a problemas como el desvanecimiento o la explosión de gradientes.

**Ejercicio 3:** Retropropagación con regularización

Añade L2 (ridge) o L1 (lasso) regularización a la red neuronal que creaste en el ejercicio 1. Entrena la red en un conjunto de datos más complejo que introduzca algo de ruido y observa si la regularización ayuda a mejorar la generalización del modelo.

**Ejercicio 4:** Experimentando con tamaños de batch

Implementa la retropropagación utilizando diferentes tamaños de batch: desde el descenso de gradiente estocástico (un ejemplo a la vez) hasta el descenso de gradiente por lotes (usando todo el conjunto de datos a la vez). Analiza cómo cambian la estabilidad y la velocidad de convergencia del entrenamiento con los diferentes tamaños de batch. Considera incluir una visualización de la disminución de la pérdida a lo largo de las iteraciones para cada configuración de tamaño de batch.

**Ejercicio 5:**  Optimización de Hiperparámetros

Utiliza la red neuronal del ejercicio 1 y experimenta con diferentes tasas de aprendizaje y números de épocas de entrenamiento. Implementa una búsqueda de cuadrícula o aleatoria para encontrar la combinación óptima de tasa de aprendizaje y número de épocas. Evalúa el rendimiento del modelo en un conjunto de validación para determinar la mejor configuración de hiperparámetros.

In [ ]:
## Tus respuestas

## Backpropagation Through Time (BPTT)

Intentaremos explicar BPTT, un algoritmo de retropropagación utilizado para entrenar redes neuronales recurrentes (RNN). Mostraremos la derivación matemática así como el código de implementación. Para el código, usaremos PyTorch para poder comparar los gradientes calculados automáticamente por PyTorch con los calculados manualmente. 

En primer lugar, definamos matemáticamente lo que hace una RNN en cada paso de tiempo $t$.

Sea $x_t \in \mathbb{R}^{d_x}$ un vector de entrada en el paso de tiempo $t$. Una RNN de tamaño $d_h$ calcula su estado oculto $h_t$ como:

$$
\begin{align*}
  r_t &= U x_t + V h_{t-1} \\
  h_t &= f \left( r_t \right)
\end{align*}
$$

donde $U \in \mathbb{R}^{d_h \times d_x}$ y $V \in \mathbb{R}^{d_h \times d_h}$ son parámetros, y $f$ es una función de activación. Nota que no incluimos el término de sesgo aquí por simplicidad.

Un vector de salida $y_t \in \mathbb{R}^{d_o}$ entonces puede ser calculado como:

$$
\begin{align*}
  s_t &= W h_t \\
  y_t &= g \left( s_t \right)
\end{align*}
$$

donde $W \in \mathbb{R}^{d_o \times d_h}$ es un parámetro y $g$ es una función de salida no lineal, que depende de la tarea (por ejemplo, softmax para clasificación). Aquí también no incluimos el término de sesgo por simplicidad. Nota que $U$, $V$, y $W$ se comparten a lo largo de los pasos de tiempo, es decir, no hay matrices separadas para cada paso de tiempo.


### Proceso

Sea $L = \mathcal{L}(y_t)$ la pérdida de nuestra RNN. Uno podría intentar derivar los gradientes de $U$, $V$, $W$ de manera ingenua. Por ejemplo, para calcular el gradiente sobre $W$:

$$
\frac{\partial L}{\partial W_{ij}} = \sum_k \frac{\partial L}{\partial (s_t)_k} \frac{\partial (s_t)_k}{\partial W_{ij}}
$$

Dado que

$$
\frac{\partial (s_t)_k}{\partial W_{ij}} =
\begin{cases}
0 & k \neq i \\
(h_t)_j & \text{de lo contrario}
\end{cases}
$$

tenemos

$$
\frac{\partial L}{\partial W_{ij}} = \frac{\partial L}{\partial (s_t)_i} (h_t)_j
$$

O, en notación matricial

$$
\overline{W} = \overline{s}_t h_t^{\top}
$$

donde la notación de barra denota el gradiente de \( L \) con respecto a él, es decir, $\overline{A}_{ij} = \frac{\partial L}{\partial A_{ij}}$ y $\overline{a}_i = \frac{\partial L}{\partial a_i}$.

Sin embargo, esto es **incorrecto**. Dado que $W$ se comparte a lo largo de los pasos de tiempo, contribuye a $s_t$ para *todos* los pasos de tiempo  $t$. Por lo tanto, al calcular el gradiente, necesitamos sumar los gradientes de todos los pasos de tiempo. En otras palabras, lo que necesitamos es:

$$
\overline{W} = \sum_t \overline{s}_t h_t^{\top}
$$

Los gradientes sobre $U$ y $V$ se pueden calcular de manera similar, es decir

$$
\begin{align*}
  \overline{U} &= \sum_t \overline{r}_t x_t^{\top} \\
  \overline{V} &= \sum_t \overline{r}_t h_{t-1}^{\top}
\end{align*}
$$

Ahora, los últimos gradientes. Gradiente sobre $r_t$ es

$$
\begin{align*}
  \frac{\partial L}{\partial (r_t)_i}
  &= \sum_j \frac{\partial L}{\partial (h_t)_j} \frac{\partial (h_t)_j}{\partial (r_t)_i} \\
  &= \frac{\partial L}{\partial (h_t)_i} f' (r_t)_i
\end{align*}
$$

O, en notación vectorial

$$
\overline{r}_t = \overline{h}_t * f' (r_t)
$$

De manera similar, el gradiente sobre $s_t$ se puede mostrar como $\overline{s}_t = \overline{y}_t * g' (s_t)$.

Por último, necesitamos calcular $\overline{h}_t$. Note que $h_t$ contribuye a $s_t$, y si $t$ no es el último paso de tiempo, entonces también contribuye a $r_{t+1}$. Por lo tanto, si $T$ denota el último paso de tiempo, entonces

$$
\begin{align*}
  \frac{\partial L}{\partial (h_T)_i}
  &= \sum_j \frac{\partial L}{\partial (s_T)_j} \frac{\partial (s_T)_j}{\partial (h_T)_i} \\
  &= \sum_j \frac{\partial L}{\partial (s_T)_j} W_{ji}
\end{align*}
$$

y para $t < T$

$$
\begin{align*}
  \frac{\partial L}{\partial (h_t)_i}
  &= \sum_j \frac{\partial L}{\partial (s_t)_j} \frac{\partial (s_t)_j}{\partial (h_t)_i}
  + \sum_k \frac{\partial L}{\partial (r_{t+1})_k} \frac{\partial (r_{t+1})_k}{\partial (h_t)_i} \\
  &= \sum_j \frac{\partial L}{\partial (s_t)_j} W_{ji}
  + \sum_k \frac{\partial L}{\partial (r_{t+1})_k} V_{ki}
\end{align*}
$$

En notación matricial (y combinando los dos)

$$
\overline{h}_t = W^{\top} \overline{s}_t +
\begin{cases}
0 & t = T \\
V^{\top} \overline{r}_{t+1} & \text{de lo contrario}
\end{cases}
$$


Así, los gradientes completos son

$$
\begin{align*}
  \overline{U} &= \sum_t \overline{r}_t x_t^{\top} \\
  \overline{V} &= \sum_t \overline{r}_t h_{t-1}^{\top} \\
  \overline{W} &= \sum_t \overline{s}_t h_t^{\top} \\
  \overline{r}_t &= \overline{h}_t * f' \left( r_t \right) \\
  \overline{s}_t &= \overline{y}_t * g' \left( s_t \right) \\
  \overline{h}_t &= W^{\top} \overline{s}_t +
    \begin{cases}
    0 & t = T \\
    V^{\top} \overline{r}_{t+1} & \text{de lo contrario}
    \end{cases}
\end{align*}
$$

### Implementacion en PyTorch

La implementación del cálculo hacia adelante es bastante directa. Comenzamos desde el paso de tiempo 1, y calculamos $r_t$, $h_t$, $s_t$ y $y_t$, dados los inputs $x_t$ y $h_{t-1}$, hasta el último paso de tiempo $T$. Comenzamos desde el paso de tiempo 1 porque en la relación de recurrencia, para calcular $r_t$ necesitamos $h_{t-1}$. Por lo tanto, el paso de tiempo anterior debe completarse primero.

La implementación del cálculo hacia atrás es similar, pero en cambio comenzamos desde el último paso de tiempo $T$ hasta el primero. Esto se debe a que en la relación de recurrencia, necesitamos $\overline{r}_{t+1}$ para calcular $\overline{h}_t$. Así, el siguiente paso de tiempo debe completarse de antemano. Además, a medida que iteramos, necesitamos acumular los gradientes en $U$, $V$ y $W$.

Primero, vamos a crear una clase RNNCell. Esta clase es responsable de un solo paso de tiempo. Tiene métodos forward y backward que realizan el cálculo hacia adelante y hacia atrás, respectivamente.

In [ ]:
import torch

In [ ]:
class RNNCell:
    """Una célula RNN responsable de un solo paso de tiempo.

    Args:
        inp_sz (int): Tamaño de la entrada.
        hid_sz (int): Tamaño del estado oculto.
        out_sz (int): Tamaño de la salida.
    """
    def __init__(self, inp_sz, hid_sz, out_sz):
        self.inp_sz = inp_sz
        self.hid_sz = hid_sz
        self.out_sz = out_sz

        # U, V, W son los parámetros, por lo que establecemos requires_grad=True
        # para indicar a PyTorch que necesitamos calcular los gradientes
        self.U = torch.empty(hid_sz, inp_sz, requires_grad=True)
        self.V = torch.empty(hid_sz, hid_sz, requires_grad=True)
        self.W = torch.empty(out_sz, hid_sz, requires_grad=True)

        # Estos son los gradientes en U, V y W que calcularemos
        # manualmente. También los compararemos con
        # los gradientes calculados por PyTorch para ver si nuestro
        # cálculo de gradientes es correcto.
        self.U_grad = torch.zeros_like(self.U)
        self.V_grad = torch.zeros_like(self.V)
        self.W_grad = torch.zeros_like(self.W)
        
        self.reset_parameters()

    def reset_parameters(self):
        """Inicializar parámetros.

        Los parámetros se inicializarán desde la distribución uniforme U(-0.1, 0.1).
        """
        s = 0.1  # un valor mayor puede hacer que los gradientes exploten
        torch.nn.init.uniform_(self.U, -s, s)
        torch.nn.init.uniform_(self.V, -s, s)
        torch.nn.init.uniform_(self.W, -s, s)
    def zero_grad(self):
        """Se pone el gradiente a cero"""
        self.U_grad.zero_()
        self.V_grad.zero_()
        self.W_grad.zero_()
    
    def forward(self, x, hp):
        """Realizar el cálculo hacia adelante.
        
        Args:
            x (Tensor): Entrada en el paso de tiempo actual.
            hp (Tensor): Estado oculto en el paso de tiempo anterior.
            
        Returns:
            Tensor: Salida en el paso de tiempo actual.
            Tensor: Estado oculto en el paso de tiempo actual.
        """
        _, h, _, y = self._get_internals(x, hp)
        return y, h

    def backward(self, y_grad, rn_grad, x, hp):
        """Realizar el cálculo hacia atrás.
        
        Args:
            y_grad (Tensor): Gradiente sobre la salida en el paso de tiempo actual.
            rn_grad (Tensor): Gradiente sobre el vector r en el siguiente paso de tiempo.
            x (Tensor): Entrada en el paso de tiempo actual que se pasó a `forward`.
            hp (Tensor): Estado oculto en el paso de tiempo anterior que se pasó a `forward`.
            
        Returns:
            Tensor: Gradiente sobre el vector r en el paso de tiempo actual.
        """
        # Obtener los vectores internos r, h, y s del cálculo hacia adelante
        r, h, s, _ = self._get_internals(x, hp)

        s_grad = y_grad * torch.sigmoid(s) * (1-torch.sigmoid(s))
        h_grad = self.W.t().matmul(s_grad) + self.V.t().matmul(rn_grad)
        r_grad = h_grad * torch.sigmoid(r) * (1-torch.sigmoid(r))

        # Los gradientes de los parámetros se acumulan
        self.U_grad += r_grad.view(-1, 1).matmul(x.view(1, -1))
        self.V_grad += r_grad.view(-1, 1).matmul(hp.view(1, -1))
        self.W_grad += s_grad.view(-1, 1).matmul(h.view(1, -1))

        return r_grad

    def _get_internals(self, x, hp):
        # Estos son los calculos forward reales
        r = self.U.matmul(x) + self.V.matmul(hp)
        h = torch.sigmoid(r)
        s = self.W.matmul(h)
        y = torch.sigmoid(s)
        
        return r, h, s, y


Ahora, creemos una clase RNN. Esta clase acepta una RNNCell y es responsable de realizar la iteración sobre los pasos de tiempo.

In [ ]:
class RNN:
    def __init__(self, cell):
        self.cell = cell
    
    def forward(self, xs, h0):
        """Realiza la computación hacia adelante para todos los pasos de tiempo.
        
        Args:
            xs (Tensor): Tensor 2-D de entradas para cada paso de tiempo. La
                primera dimensión corresponde al número de pasos de tiempo.
            h0 (Tensor): Estado oculto inicial.
            
        Returns:
            Tensor: Tensor 2-D de salidas para cada paso de tiempo. La primera
                dimensión corresponde al número de pasos de tiempo.
            Tensor: Tensor 2-D de estados ocultos para cada paso de tiempo más
                `h0`. La primera dimensión corresponde al número de pasos de
                tiempo.
        """
        ys, hs = [], [h0]  # Inicializa listas vacías para almacenar las salidas y los estados ocultos
        for x in xs:
            y, h = self.cell.forward(x, hs[-1])  # Calcula la salida y el siguiente estado oculto
            ys.append(y)  # Agrega la salida a la lista de salidas
            hs.append(h)  # Agrega el estado oculto al final de la lista de estados ocultos
        return torch.stack(ys), torch.stack(hs)  # Apila las salidas y los estados ocultos en tensores
    
    def backward(self, ys_grad, xs, hs):
        """Realiza la computación hacia atrás para todos los pasos de tiempo.
        
        Args:
            ys_grad (Tensor): Tensor 2-D de los gradientes en las salidas
                para cada paso de tiempo. La primera dimensión corresponde a
                el número de pasos de tiempo.
            xs (Tensor): Tensor 2-D de entradas para cada paso de tiempo que
                fue pasado a `forward`.
            hs (Tensor): Tensor 2-D de estados ocultos que es devuelto por
                `forward`.
        """
        # Para el último paso de tiempo, el gradiente en r es cero
        rn_grad = torch.zeros(self.cell.hid_sz)  # Inicializa el gradiente de la celda recurrente como cero

        for y_grad, x, hp in reversed(list(zip(ys_grad, xs, hs))):
            rn_grad = cell.backward(y_grad, rn_grad, x, hp)  # Calcula el gradiente para cada paso de tiempo


A continuación, la función de pérdida. Aquí utilizamos una pérdida simple de suma de cuadrados.

In [ ]:
def compute_loss(ys, ts):
    return 0.5 * torch.sum((ys - ts)**2)

Ahora, construimos nuestra RNN. Tendrá un tamaño de entrada de 2, un tamaño oculto de 3 y un tamaño de salida de 4.

In [ ]:
cell = RNNCell(2, 4, 5)
rnn = RNN(cell)

Luego, creamos entradas y objetivos ficticios para nuestra RNN. Estableceremos el número de pasos de tiempo en 3.

In [ ]:
xs = torch.rand(3, cell.inp_sz)
hp = torch.rand(cell.hid_sz)
ts = torch.rand(3, cell.out_sz)

A continuación, realizamos la computación del forward y calculamos la pérdida.

In [ ]:
ys, hs = rnn.forward(xs, hp)
loss = compute_loss(ys, ts)

In [ ]:
loss


Veamos los gradientes calculados por PyTorch.


In [ ]:
loss.backward()

In [ ]:
rnn.cell.U.grad

In [ ]:
rnn.cell.V.grad

In [ ]:
rnn.cell.W.grad

Ahora, ejecutemos nuestra computación backward y comparemos el resultado con el de PyTorch.

In [ ]:
# Esto se obtiene de nuestra función de pérdida de suma de cuadrados
ys_grad = ys - ts

with torch.no_grad():  # necesario para que PyTorch no genere un erro
    rnn.cell.zero_grad()
    rnn.backward(ys_grad, xs, hs)

Ahora veamos si nuestros gradientes calculados manualmente son correctos, es decir, si son iguales a los de PyTorch.

In [ ]:
rnn.cell.U_grad

In [ ]:
rnn.cell.V_grad

In [ ]:
rnn.cell.W_grad


¡Y son iguales! Así que nuestra computación es realmente correcta.

### Ejercicios

**Ejercicio 1**: Derivación Manual de BPTT

Dado un simple RNN con una capa oculta y una función de activación sigmoide, realiza la derivación manual de los gradientes para los parámetros $U$, $V$ y $W$ utilizando BPTT. Asume que la RNN tiene una sola unidad oculta y procesa secuencias de tres tiempos.

**Ejercicio 2**: Implementación de BPTT desde cero

Implementa una RNN simple en Python sin usar bibliotecas de deep learning como TensorFlow o PyTorch. Tu RNN debe incluir métodos de forward y backward, y debe ser capaz de procesar secuencias de longitud variable.

**Ejercicio 3**: Comparación de gradientes

Utiliza PyTorch para crear una RNN simple y compara los gradientes que calcula automáticamente con los que calculaste manualmente en el ejercicio 1 o implementaste en el ejercicio 2.

**Ejercicio 4:** Exploración de la explosión y desvanecimiento de gradientes

Modifica la RNN implementada en el ejercicio 2 para incluir secuencias de entrada de diferentes longitudes. Experimenta con inicializaciones de parámetros y observa cómo afectan a la magnitud de los gradientes durante el entrenamiento.

**Ejercicio 5:** BPTT con diferentes funciones de activación

Implementa varias funciones de activación (ReLU, tanh, sigmoide) en tu RNN del ejercicio 2. Entrena tu modelo en un conjunto de datos de prueba simple (puede ser generado sintéticamente) y compara cómo la elección de la función de activación afecta el rendimiento.



In [ ]:
## Tus respuestas